<a href="https://colab.research.google.com/github/EnricCruzadoCampos/CounterSeedsGO/blob/main/notebooks/APP_CONTADOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🌾 ANALIZADOR MULTI-CLASE PRO (12 Tipos)
# @markdown Detecta: Trigo (Sano, Roto, Delgado, Cáscara), Arroz, Maíz, Judía, Paja...

import os
import sys
import cv2
import numpy as np
import urllib.request
import ipywidgets as widgets
from IPython.display import display, Image
from ultralytics import YOLO

# --- CONFIGURACIÓN ---
USUARIO = "EnricCruzadoCampos"
REPO = "CounterSeedsGO"
RAMA = "main"
URL = f"https://github.com/{USUARIO}/{REPO}/raw/{RAMA}/models/best.pt"

# --- MAPA DE CLASES (Basado en tu último log) ---
CLASES = {
    0:  {"nombre": "Judía (Bean)",       "color": (255, 100, 100)}, # Rosa
    1:  {"nombre": "Maíz (Corn)",        "color": (0, 255, 255)},   # Amarillo
    2:  {"nombre": "Impureza Genérica",  "color": (50, 50, 50)},    # Gris oscuro
    3:  {"nombre": "Arroz",              "color": (255, 191, 0)},   # Azul cielo
    4:  {"nombre": "Arroz Integral",     "color": (165, 42, 42)},   # Marrón
    5:  {"nombre": "Arroz Molido",       "color": (200, 200, 200)}, # Gris claro
    6:  {"nombre": "Paja (Straw)",       "color": (0, 165, 255)},   # Naranja
    7:  {"nombre": "Trigo Roto",         "color": (0, 0, 255)},     # ROJO (Defecto)
    8:  {"nombre": "Cáscara (Chaff)",    "color": (0, 140, 255)},   # Naranja claro
    9:  {"nombre": "Trigo Sano",         "color": (0, 255, 0)},     # VERDE
    10: {"nombre": "Trigo Delgado",      "color": (0, 0, 150)},     # Rojo oscuro
    11: {"nombre": "Xian",               "color": (255, 0, 255)}    # Magenta
}

# --- SISTEMA ---
def setup_system():
    try: import ultralytics
    except: os.system('pip install ultralytics opencv-python-headless > /dev/null')

setup_system()

if os.path.exists('best.pt'): os.remove('best.pt')
try: urllib.request.urlretrieve(URL, 'best.pt')
except: print("❌ Error descargando modelo.")

print("⚙️ Cargando modelo YOLOv11 (12 Clases)...")
model = YOLO('best.pt')

# --- APP INTERACTIVA ---
print("\n" + "="*50)
print("   📸 SUBE TU FOTO PARA ANALIZAR")
print("="*50)
uploaded = files.upload()

if not uploaded:
    print("⚠️ No subiste nada.")
else:
    nombre = list(uploaded.keys())[0]
    file_bytes = np.frombuffer(uploaded[nombre], np.uint8)
    img_orig = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    # Pre-cálculo
    results_cache = model.predict(img_orig, conf=0.1, iou=0.5, verbose=False)
    detecciones = results_cache[0].boxes.data.cpu().numpy()

    def update(conf, iou, tam_min, tam_max):
        # Re-filtrado manual rápido (simulando NMS si fuera necesario, pero aquí filtramos visualmente)
        # Nota: Para aplicar NMS real al cambiar IOU, deberíamos re-predecir,
        # pero para velocidad usaremos el filtrado de confianza sobre la predicción base.

        # Si el usuario cambia IOU drásticamente, recalculamos NMS
        # (Pequeño truco para no llamar a model.predict cada milisegundo)
        if abs(update.last_iou - iou) > 0.05:
             update.res = model.predict(img_orig, conf=0.05, iou=iou, verbose=False)
             update.dets = update.res[0].boxes.data.cpu().numpy()
             update.last_iou = iou

        img = img_orig.copy()
        cnt = {k:0 for k in CLASES}

        for box in update.dets:
            x1, y1, x2, y2, sc, cls_id = box
            if sc < conf: continue

            area = (x2-x1)*(y2-y1)
            if area < tam_min or area > tam_max: continue

            cls_id = int(cls_id)
            if cls_id in CLASES:
                cnt[cls_id] += 1
                color = CLASES[cls_id]['color']
                cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), color, 2)

        # Panel
        h, w, _ = img.shape
        pan = np.zeros((h, 350, 3), dtype=np.uint8)
        pan[:] = (30,30,30)

        y=40
        cv2.putText(pan, "RESULTADOS", (15,y), 0, 0.8, (255,255,255), 2)
        y+=40

        # Agrupar Trigo Malo vs Bueno
        trigo_bueno = cnt[9]
        trigo_malo = cnt[7] + cnt[10] # Roto + Delgado
        impurezas = cnt[6] + cnt[8] + cnt[2] # Paja + Cáscara + Genérica
        otros = sum(cnt.values()) - (trigo_bueno + trigo_malo + impurezas)

        # Mostrar lista
        for cid, info in CLASES.items():
            if cnt[cid] > 0:
                cv2.circle(pan, (20,y-5), 6, info['color'], -1)
                cv2.putText(pan, f"{cnt[cid]} {info['nombre']}", (40,y), 0, 0.6, (200,200,200), 1)
                y+=30

        y+=20
        cv2.line(pan, (15,y), (335,y), (100,100,100), 1)
        y+=30

        # Calidad Trigo
        total_trigo = trigo_bueno + trigo_malo
        calidad = (trigo_bueno / total_trigo * 100) if total_trigo > 0 else 0
        col = (0,255,0) if calidad > 85 else (0,165,255) if calidad > 50 else (0,0,255)

        cv2.putText(pan, f"CALIDAD TRIGO: {calidad:.1f}%", (15,y), 0, 0.7, col, 2)

        # Unir
        fin = np.hstack((img, pan))
        _, jpg = cv2.imencode('.jpg', fin)
        display(Image(data=jpg))

    update.last_iou = 0.5
    update.dets = detecciones

    # Sliders
    s_conf = widgets.FloatSlider(0.4, min=0.1, max=0.9, step=0.05, description='Confianza')
    s_iou = widgets.FloatSlider(0.5, min=0.1, max=0.9, step=0.05, description='IoU (Solape)')
    s_min = widgets.IntSlider(100, min=0, max=2000, step=50, description='Min Size')

    widgets.interact(update, conf=s_conf, iou=s_iou, tam_min=s_min, tam_max=widgets.fixed(100000))

⚙️ Cargando modelo YOLOv11 (12 Clases)...

   📸 SUBE TU FOTO PARA ANALIZAR
